# **Grouping based on Geo Data**
37명 4개씩 상점 분할하기
- **Total** : 151 개
- 1 개의 상호가 2개의 점포를 소유중 **(1개로 포함하여 작업)**

## **1 Loading the table**
간단한 내용 수정하기

In [1]:
# 데이터 불러오기
import pandas as pd
file_csv = "./data/store_fin.csv"
table = pd.read_csv(file_csv)

# 중복데이터 약간씩 수정 작업 진행하기
from collections import Counter
_count_address = dict(Counter([_ for _ in table['address']]))
_count_address = {k:str(v) for k, v in _count_address.items() if v != 1}

# 중복 주소의 lon, lat 데이터 약간씩 수정
for _list in [list(table[table["address"]==_address].index)  for _address in _count_address]:
    for no, index in enumerate(_list):
        lon, lat = table.loc[index, 'lon'], table.loc[index, 'lat']
        table.loc[index, 'lon'] = lon - 0.000039 * no
        table.loc[index, 'lat'] = lat - 0.000039 * no * 1.7        
_count_address

{'서울특별시 관악구 난향길 14': '4',
 '서울특별시 관악구 난향길 12': '2',
 '서울특별시 관악구 난곡로24길 29': '3',
 '서울특별시 관악구 난곡로24길 6': '3',
 '서울특별시 관악구 난곡로26길 7': '2',
 '서울특별시 관악구 호암로22길 69 신국고시원': '2',
 '서울특별시 관악구 대학5길 21': '2',
 '서울특별시 관악구 신림로11길 20': '2',
 '서울특별시 관악구 신림로65길 31': '2',
 '서울특별시 관악구 남부순환로214길 29': '2',
 '서울특별시 관악구 봉천로 465': '2'}

## **2 Visualization**
정리된 데이터 시각화 하기
    # 'white', 'beige', 'pink', 'lightgray','lightgreen', 'lightblue','lightred',  
    # color_map = ['gray','red', 'purple', 'lightred',  'black',  'green',  'darkblue', 
    #    'blue', 'darkgreen',  'darkpurple', 'darkred', 'orange', 'cadetblue'] * 10

In [2]:
# import folium
# from folium import plugins
# def folium_map(table, title='atrist', tooltip="name", opacity=0.3, zoom=14):
#     table    = table.reset_index(drop=True)
#     map_data = folium.Map([table['lat'].mean(), table['lon'].mean()], zoom_start=zoom)
#     for _ in range(len(table)): # 개별 데이터 Spot 찍기
#         _table = table.iloc[_, :]
#         icon_style = plugins.BeautifyIcon(
#             number = str(table[title]),
#             icon_shape   = "marker",  inner_icon_style = 'margin=top:1;',
#             border_color = 'darkred', text_color = 'darkred', border_width = 2)
#         folium.Marker( [ _table['lat'], _table['lon'] ], 
#             opacity = opacity, tooltip = _table[tooltip],
#             icon = icon_style ).add_to(map_data)
#     return map_data

# temp_table = table[table["address"] == list(_count_address.keys())[2]]
# folium_map(temp_table, title='artist', tooltip='name', opacity=1, zoom=20)

## **3 Saving the files**
파일 저장하기

In [3]:
# for _list in [list(table[table["address"]==_address].index)  for _address in _count_address]:
#     for no, index in enumerate(_list):
#         lon, lat = table.loc[index, 'lon'], table.loc[index, 'lat']
#         name = table.loc[index, 'address']
#         print(name, lon, lat)

# table.columns = ['observer', 'label', 'titla', 'name', 'phone', 'lat', 'lon', 'address', 'addressraw', 'type']
# table.to_csv('./data/ourtown_store_geo_fin2.csv', index=None)

## **3 Folium Area View**
Folium Map View

In [4]:
# 데이터 불러오기
import pandas as pd
file_csv = "./data/store_fin.csv"
table = pd.read_csv(file_csv)
table = table.sort_values(by=['master','artist','lat','lon']).reset_index(drop=True)
table.head(2)

,master,artist,title,name,phone,lat,lon,address,addressraw,type
0,1,1,영진떡방앗간,박명일,010-8507-6657,37.462786,126.918001,서울특별시 관악구 난향길 14,난향길 14,떡집
1,1,1,카페비온,유민영,010-9688-7097,37.462847,126.917727,서울특별시 관악구 난향길 24 현경빌딩,난향길 24,카페


In [5]:
def set_index(table, field_name_from:str, field_name_to:str):
    result_list = []
    for _ in sorted(set(table[field_name_from].to_list())): # Artist Nunber
        for no, _ in enumerate(list(table[table[field_name_from]==_].index)): # Artist Store Number
            table.loc[_, field_name_to] = (no + 1)
    print("processing is Done..")
table.insert(2, 'store', table['artist'])
table.insert(2, 'artist_no', table['artist'])
set_index(table, 'artist', 'store')
table.tail(2)

processing is Done..


,master,artist,artist_no,store,title,name,phone,lat,lon,address,addressraw,type
148,3,37,37,3,임서하헤어마벨,임서하,010-8689-8650,37.481842,126.941729,서울특별시 관악구 장군봉1길 29,장군봉1길 29,미용업
149,3,37,37,4,정가,유영옥,010-5331-7715,37.482754,126.939874,서울특별시 관악구 남부순환로 1710 이너스빌딩,남부순환로 1710,음식점


## **Artist Middle Count**
Master 단위로 Count Table 추가하기

In [6]:
check_dict = {}
for number in [2,3]:
    check_dict[number] = {}
    for no,_ in enumerate(sorted(set(table[table['master']==number]['artist']))):
        check_dict[number][_] = (no + 1)
print(check_dict.keys())

for number in [2,3]:
    for _ in list(table[table['master']==number].index):
        table.loc[_, 'artist_no'] = check_dict[number][table.loc[_, 'artist']]
# 정리된 Table 컬럼 순서 바꾸기
table = table.loc[:, ['artist', 'master','artist_no', 'store', 'title', 'name', 'phone',
       'lat', 'lon', 'address', 'addressraw', 'type']]
table.tail(3)

dict_keys([2, 3])


,artist,master,artist_no,store,title,name,phone,lat,lon,address,addressraw,type
147,37,3,12,2,신현선무관,안현석,010-2275-5616,37.481743,126.942606,서울특별시 관악구 장군봉1길 45,"장군봉1길 45, 2층",학원
148,37,3,12,3,임서하헤어마벨,임서하,010-8689-8650,37.481842,126.941729,서울특별시 관악구 장군봉1길 29,장군봉1길 29,미용업
149,37,3,12,4,정가,유영옥,010-5331-7715,37.482754,126.939874,서울특별시 관악구 남부순환로 1710 이너스빌딩,남부순환로 1710,음식점


## **Sorting Table**
컬럼 데이터 변경 및 Artist 순서 재정렬

In [7]:
result_order = []
for _ in table.artist:
    if _ not in result_order:
        result_order.append(_)

# 순서 새롭게 정렬하기
result_dict = {}
for no, _ in enumerate(result_order):
    result_dict[_] = (no + 1)
# 새롭게 정리된 Index 추가하기
for _ in range(len(table)):
    number = table.loc[_,"artist"]
    table.loc[_,"artist"] = result_dict[number]
# table.to_csv('./data/store_fin3.csv', index=None)
table.tail(3)

,artist,master,artist_no,store,title,name,phone,lat,lon,address,addressraw,type
147,37,3,12,2,신현선무관,안현석,010-2275-5616,37.481743,126.942606,서울특별시 관악구 장군봉1길 45,"장군봉1길 45, 2층",학원
148,37,3,12,3,임서하헤어마벨,임서하,010-8689-8650,37.481842,126.941729,서울특별시 관악구 장군봉1길 29,장군봉1길 29,미용업
149,37,3,12,4,정가,유영옥,010-5331-7715,37.482754,126.939874,서울특별시 관악구 남부순환로 1710 이너스빌딩,남부순환로 1710,음식점


## **Folium Visualization**
테이블 내용을 바탕으로 지도 그리기

In [8]:
text = "B12-2"

In [9]:
text[:1] + '-' + text[1:]

'B-12-2'

In [11]:
import folium, re
from folium import plugins
def folium_map(table, title='atrist', tooltip="name", opacity=0.3, zoom=14):
    color_map = ['red','blue','purple','orange','green', 'black', 'green','darkgreen','darkpurple', 'darkred', 'orange'] * 30
    color_map = {_:color_map[no]  for no, _ in enumerate(sorted(set(table[title].to_list())))}
    color_master_map = {"1":"darkblue", "2":"orange", "3":"darkred"}
    master_text = {"1":"A","2":"B","3":"C"}
    table    = table.reset_index(drop=True)
    table[title] = [str(_)  for _ in table[title]]
    table["master"] = [str(_)  for _ in table["master"]]
    map_data = folium.Map([table['lat'].mean(), table['lon'].mean()], zoom_start=zoom)
    for _ in range(len(table)): # 개별 데이터 Spot 찍기
        _table = table.iloc[_, :]
        # title = f"({_table['master']})-({_table['artist']})"
        # title = f"{_table['master']},{_table['artist']} {}"
        master_alpha = master_text[_table['master']]
        title = f"{_table['master']},{_table['artist_no']},{_table['store']}"
        title = f"{_table['artist_no']}-{_table['store']}"
        title = f"{master_alpha}-{_table['artist_no']}-{_table['store']}"
        color_text = color_map[int(_table['artist_no'])]
        popup_text = f"{title}<br/>{_table['title']} ({_table['type']}) , {_table['name']} : {_table['phone']}<br/>{_table['address']} | {_table['addressraw']}"   
        icon_style = plugins.BeautifyIcon(
            number = title, 
            icon_shape   = "marker", 
            # inner_icon_style = 'font-size:10px;text-align:center;',
            inner_icon_style = 'font-size:13px',
            border_color = color_master_map[_table["master"]], 
            text_color = color_text, border_width = 2)
        folium.Marker( [ _table['lat'], _table['lon'] ], 
            opacity = opacity, tooltip = popup_text,
            icon = icon_style ).add_to(map_data)
    return map_data

# temp_table = table[table["address"] == list(_count_address.keys())[2]]
_map = folium_map(table, title='artist', tooltip='name', opacity=1, zoom=14)
_map.save('web/gkart_v1.html')
_map

In [12]:
import pandas as pd
file_csv = "./data/store_fin3.csv"
table = pd.read_csv(file_csv)
table.head(2)

,artist,master,artist_no,store,title,name,phone,lat,lon,address,addressraw,type
0,1,1,1,1,영진떡방앗간,박명일,010-8507-6657,37.462786,126.918001,서울특별시 관악구 난향길 14,난향길 14,떡집
1,1,1,1,2,카페비온,유민영,010-9688-7097,37.462847,126.917727,서울특별시 관악구 난향길 24 현경빌딩,난향길 24,카페


In [13]:
import folium, re
from folium import plugins
from folium.features import DivIcon
def folium_map(table, title='atrist', zoom=14):
    """rCircle Rounded Map Marker"""
    map_data = folium.Map([table['lat'].mean(), table['lon'].mean()], zoom_start=zoom)
    master_text, table = {"1":"A", "2":"B","3":"C"}, table.reset_index(drop=True)
    color_master_map = {"1":"darkblue","2":"orange","3":"darkred"}
    color_map = ['blue','purple','darkred','green',
                 'black'] * 30  # 'darkgreen','darkpurple',
    table["master"] = [str(_)  for _ in table["master"]]
    for _ in range(len(table)): # 개별 데이터 Spot 찍기
        _table = table.iloc[_, :]
        # print(_table['artist_no'], type(_table['artist_no']))
        color_text = color_map[_table['artist_no']]
        title  = f"{_table['artist_no']}-{_table['store']}"
        manager = f"{master_text[_table['master']]}"
        popup_text = f"""{title}<br/>
            {_table['title']} ({_table['type']}) , {_table['name']} : {_table['phone']}<br/>
            {_table['address']} | {_table['addressraw']}"""
        # Text Div ICON 내용 정의하기
        marker_sytle = DivIcon(
            icon_size = (150,50),
            icon_anchor = (7,20),
            html = f'''<div style="font-size:15px;color:{color_text};
            font-weight:bold;margin-top:20px;margin-left:15px">{title}</div>'''
        )
        # Marker Icon Style 정의하기
        icon_style = plugins.BeautifyIcon(
            number = master_text[_table['master']], # + "-" + title, # , 
            icon_shape   = "circle", 
            inner_icon_style = 'font-size:13px',
            border_color = color_master_map[_table["master"]], 
            text_color = color_text, 
            border_width = 2)
        
        folium.Marker(
            [ _table['lat'], _table['lon']],
            tooltip = popup_text,
            icon = icon_style).add_to(map_data)
        map_data.add_child(folium.Marker(
            [_table['lat'], _table['lon']],
            tooltip = popup_text,
            icon =marker_sytle,
        ))
    return map_data

# temp_table = table[table["address"] == list(_count_address.keys())[2]]
_map = folium_map(table, title='artist', zoom=14)
_map.save('web/gkart_v1.html')
_map

In [ ]:
temp_table = table[table["address"] == list(_count_address.keys())[2]]
temp_table.loc[:,['lat','lon']].values

%matplotlib inline
# map_data = folium.Map([temp_table['lat'].mean(), temp_table['lon'].mean()], zoom_start=15)
# plugins.MarkerCluster(temp_table.loc[:,['lat','lon']].values).add_to(map_data)
# map_data

In [ ]:
import numpy as np
N = 100 
data = np.array( 
    [ np.random.uniform(low=35, high=60, size=N), # Random latitudes in Europe. 
     np.random.uniform(low=-12, high=30, size=N), # Random longitudes in Europe. 
    ] ).T 
popups = [str(i) for i in range(N)] # Popups texts are simple numbers. 
m = folium.Map([45, 3], zoom_start=4) 
plugins.MarkerCluster(data, popups=popups).add_to(m) 
# m.save(os.path.join('results', 'Plugins_1.html')) 
m

In [ ]:
import folium
from folium.features import DivIcon

m = folium.Map(
    location=[45.3288, -121.6625],
    zoom_start=12,
    #tiles='Mapbox Bright'
)

p1 = [45.3288, -121.6625]
folium.Marker(p1, icon=DivIcon(
        icon_size=(150,36),
        #icon_anchor=(7,20),
        icon_anchor=(7,5),
        html='<div style="font-size:12pt;color:black;align-items:center">A-12-1</div>',
        )).add_to(m)
m.add_child(folium.CircleMarker(p1, radius=15))

p2 = [45.3311, -121.7113]
folium.Marker(p2, 
        tooltip="text0",
        icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(7,20),
        html='<div style="font-size: 18pt; color : black">2</div>',
        )).add_to(m)
m.add_child(folium.CircleMarker(p2, radius=15))